In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model
#from sklearn.model_selection import training_test_split
#from sklean
import os 
import pandas as pd
import utils

In [2]:
X, supply, demand = utils.LoadDataset('./training_set/DesignMatrices/')
X.shape, supply.shape, demand.shape

X=X[:, :2]

In [3]:
Xtrain, supplyTrain, demandTrain, Xval, supplyVal, demandVal = utils.SplitByRegion(X, supply, demand, .7)

In [4]:
print 'Xtrain = {}, demandTrain = {}, supplyTrain = {}'.format(Xtrain.shape, demandTrain.shape, supplyTrain.shape)
print 'Xval = {}, demandVal = {}, supplyVal = {}'.format(Xval.shape, demandVal.shape, supplyVal.shape)

Xtrain = (99981L, 2L), demandTrain = (99981L,), supplyTrain = (99981L,)
Xval = (42899L, 2L), demandVal = (42899L,), supplyVal = (42899L,)


In [5]:
def getMAE(truths, preds): 
    return (np.mean(np.abs(truths-preds)))

In [6]:
bestscore = np.inf
bestmodel, bestdeg = None, None
for d in np.arange(1,11):
    Xnew=PolynomialFeatures(degree=d).fit_transform(Xtrain)
    model=linear_model.LinearRegression()
    model.fit(Xnew, demandTrain-supplyTrain)
    
    ypreds=model.predict(PolynomialFeatures(degree=d).fit_transform(Xval))
    print 'degree = {}\tMAE={}'.format(d, getMAE(demandVal-supplyVal, ypreds))
    
    if (bestscore > getMAE(demandVal-supplyVal, ypreds)):
        bestscore = getMAE(demandVal-supplyVal, ypreds)
        bestmodel = model
        bestdeg = d

print bestdeg 

degree = 1	MAE=12.3902010981
degree = 2	MAE=12.2988563898
degree = 3	MAE=12.3946963212
degree = 4	MAE=12.7413107408
degree = 5	MAE=12.7276542744
degree = 6	MAE=12.7164578129
degree = 7	MAE=13.3611888763
degree = 8	MAE=12.9508606378
degree = 9	MAE=13.0784254868
degree = 10	MAE=13.4786441457
2


In [13]:
fnames = [x for x in sorted(os.listdir('./test_set/DesignMatrices/')) if x[0]!='.']
for fname in fnames: 
    print fname 
    Xtestpd = pd.read_csv('./test_set/DesignMatrices/'+fname)
    Xtestpd = Xtestpd[Xtestpd.Timeslot <= 51]
    Xtest = Xtestpd[['RegionID', 'Timeslot']].values
    if Xtest.size ==0:
        continue
    Ytest = bestmodel.predict(PolynomialFeatures(degree=bestdeg).fit_transform(Xtest))
    
    myout = pd.DataFrame(Ytest, columns=['gap'])
    myout['district_date_slot'] = Xtestpd['RegionHash'].values
    myout['date'] = fname.split('.')[0]
    #myout
    myout['district_date_slot'] = myout.district_date_slot.astype(str).str.cat(Xtestpd.Date.astype(str), sep='_')
    myout['district_date_slot'] = myout.district_date_slot.astype(str).str.cat(Xtestpd.Timeslot.astype(str), sep='-')
    #myout=myout.set_index('district_date_slot')
    
    kaggleout = pd.read_csv('sample.csv', index_col='district_date_slot')
    kaggleout.ix[myout['district_date_slot'].values] = myout['gap'].values[:, np.newaxis]
    #kaggleout.loc[myout['district_date_slot'].values].gap = myout.gap.values
    kaggleout.to_csv('sample.csv')

2016-01-23.csv
2016-01-25.csv
2016-01-27.csv
2016-01-29.csv
2016-01-31.csv


In [144]:
kaggleout[kaggleout.gap!=0]

,gap
district_date_slot,
90c5a34f06ac86aee0fd70e2adce7d8a_2016-01-23-43,8.660027
f2c8c4bb99e6377d21de71275afd6cd2_2016-01-23-43,8.839839
58c7a4888306d8ff3a641d1c0feccbe3_2016-01-23-43,9.012416
b26a240205c852804ff8758628c0a86a_2016-01-23-43,9.177757
4b9e4cf2fbdc8281b8a1f9f12b80ce4d_2016-01-23-43,9.335863
1cbfbdd079ef93e74405c53fcfff8567_2016-01-23-43,9.486734
929ec6c160e6f52c20a4217c7978f681_2016-01-23-43,9.630370
82cc4851f9e4faa4e54309f8bb73fd7c_2016-01-23-43,9.766770
b702e920dcd2765e624dc1ce3a770512_2016-01-23-43,9.895935
